In [4]:
from astropy.io import fits
from astropy.table import Table,join
import numpy as np
import pylab as plt
import random
from scipy import stats
from sklearn.neighbors import KDTree
import time
from sklearn.metrics import mean_squared_error
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask
from LLR import LLR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from tensorflow import keras
from tensorflow.keras import layers
import xgboost as xgb
from LLR import LLR
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [2]:
## DATA ##
## I'm combining fastphot,fastspect, and ztile to make sure I use the same data everywhere ##

zall_path = "/global/cfs/cdirs/desi/spectro/redux/fuji/zcatalog/ztile-sv1-bright-cumulative.fits"
data1 = Table.read(zall_path,hdu=1)
needed1 = ["TARGETID", "SV1_BGS_TARGET", "SPECTYPE", "DELTACHI2", "Z", "ZWARN", "FIBER", "PETAL_LOC", "TILEID"]

fastspec_path = "/global/cfs/cdirs/desi/spectro/fastspecfit/fuji/catalogs/fastspec-fuji-sv1-bright.fits"
data2 = Table.read(fastspec_path,hdu=1)
data2.rename_column('CONTINUUM_COEFF', 'CONTINUUM_COEFF_FASTSPEC')
data2.rename_column('CONTINUUM_AV', 'CONTINUUM_AV_FASTSPEC')

needed2 = ["TARGETID", "OII_3726_EW", "OII_3729_EW", "HGAMMA_EW", "HBETA_EW", "OIII_4959_EW", "OIII_5007_EW", "NII_6548_EW", "HALPHA_EW", "NII_6584_EW", "SII_6716_EW", "SII_6731_EW",\
           "FLUX_SYNTH_G", "FLUX_SYNTH_R", "FLUX_SYNTH_Z", 'CONTINUUM_COEFF_FASTSPEC', 'CONTINUUM_AV_FASTSPEC',\
           "OII_3726_EW_IVAR", "OII_3729_EW_IVAR", "HGAMMA_EW_IVAR", "HBETA_EW_IVAR", "OIII_4959_EW_IVAR", "OIII_5007_EW_IVAR", "NII_6548_EW_IVAR", "HALPHA_EW_IVAR", "NII_6584_EW_IVAR",\
           "SII_6716_EW_IVAR", "SII_6731_EW_IVAR"]


fastphot_path = "/global/cfs/cdirs/desi/spectro/fastspecfit/fuji/catalogs/fastphot-fuji-sv1-bright.fits"
data3 = Table.read(fastphot_path,hdu=1)
data3.rename_column('CONTINUUM_COEFF', 'CONTINUUM_COEFF_FASTPHOT')
data3.rename_column('CONTINUUM_AV', 'CONTINUUM_AV_FASTPHOT')

needed3 = ["TARGETID", "ABSMAG_SDSS_U", "ABSMAG_SDSS_G", "ABSMAG_SDSS_R", "ABSMAG_SDSS_I", "ABSMAG_SDSS_Z", "ABSMAG_W1", 'CONTINUUM_COEFF_FASTPHOT', 'CONTINUUM_AV_FASTPHOT']

data4 = join(data1[needed1], data2[needed2], keys="TARGETID")
data = join(data4, data3[needed3], keys="TARGETID")

N=len(data['TARGETID'])

## Adding the sum of OII doublets to use them as a single line
data.add_column(data["OII_3726_EW"]+data["OII_3729_EW"], name='OII_DOUBLET_EW')
data.add_column(1/(data["OII_3726_EW_IVAR"]+data["OII_3729_EW_IVAR"]), name='OII_DOUBLET_EW_IVAR')

not_used, ind = np.unique(data['TARGETID'], return_index=True)
data = data[ind]

/tmp/ipykernel_6800/4145400336.py:33: RuntimeWarning: divide by zero encountered in true_divide
  data.add_column(1/(data["OII_3726_EW_IVAR"]+data["OII_3729_EW_IVAR"]), name='OII_DOUBLET_EW_IVAR')


In [5]:
server = 1 # 0 is perlmutter, 1 is cori
server_paths = ['/pscratch/sd/a/ashodkh/', '/global/cscratch1/sd/ashodkh/']
N=len(data['TARGETID'])

## Selecting data and doing LLR to predict lines ##
lines=["OII_DOUBLET_EW","HGAMMA_EW","HBETA_EW","OIII_4959_EW","OIII_5007_EW","NII_6548_EW","HALPHA_EW"\
       ,"NII_6584_EW","SII_6716_EW","SII_6731_EW"]

magnitude_names=["ABSMAG_SDSS_U", "ABSMAG_SDSS_G", "ABSMAG_SDSS_R", "ABSMAG_SDSS_I", "ABSMAG_SDSS_Z"]
#magnitude_names = ['FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'Z']
 
n=30*10**3 # initial selection size. Should be smaller later after selecting flux>0 from raw data to make sure same data is used.
 
    
# calculating min and max redshift to have de-redshifted wavelengths be in the interval w1,w2 A
w1=3400
w_min=3600
z_min=w_min/w1-1
w2 = 8500
w_max = 9824
z_max = 0.3


## I need to mimick target-selection-only to get same data

select = ((data["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT"))>0)*(data["SPECTYPE"]=="GALAXY")*(data["DELTACHI2"]>=25)\
         *(data["Z"]>z_min)*(data["Z"]<z_max)*(data["ZWARN"]==0)

select_size = len(np.where(select)[0])
print(select_size)
data_select = data[select]
#select_half = int(select_size/2)
select_half = select_size - 1
data_train = data_select[:select_half]
data_test = data_select[select_half:]

N = len(data_train['TARGETID'])
snr_cut = 1
# calculating snr for all lines and setting the snr cut boolean as select_snr
snr_all = np.zeros([N,len(lines)])
snr_all[:,0] = data_train[lines[0]]*np.sqrt(data_train[lines[0]+"_IVAR"])

cut_Halpha = True # cut Halpha on all data and later cut snr>0 for every line separately
cut_all = False # cut on all lines together, which extremely restricts the data
for i in range(1,len(lines)):
    snr_all[:,i]=data_train[lines[i]]*np.sqrt(data_train[lines[i]+"_IVAR"])
    if cut_all:
        select_snr = select_snr*(snr_all[:,i]>snr_cut)
if cut_Halpha:
    select_snr = snr_all[:,6]>snr_cut
    
#parameters
l = 6
N = 6
run = 0
loga = True     # if true then predicts log(EW)
m = 6           # model index. 0 is LLR, 1 is RandomForest, 2 is GradientBoosting from sklearn, 3 is XGboost, 4 is neural network

# adding snr cuts to target selection
select = (select_snr)*(snr_all[:,l]>0)
print(len(np.where(select)[0]))
target_ids = data_train["TARGETID"][select]
target_ids2 = np.load(server_paths[server] + "target_selection/target_ids_selection" + str(run) + "_" + str(lines[l]) + ".txt.npz")['arr_0']

target_pos = np.where(select)[0][:n] 

# flux cut after initial target selection and taking first n data
n = 23*10**3
target_pos = target_pos[:n]

# assigning features as colors and standardizing them. I also add ones to include the y-intercept as part of the parameter matrix if m==0 (LLR).
magnitudes_s = data_train[magnitude_names][target_pos]  
magnitudes = np.zeros([n,len(magnitude_names)])
for j in range(len(magnitude_names)):
    magnitudes[:,j] = magnitudes_s[magnitude_names[j]][:n]

ones = np.ones([n,1])
scalar = StandardScaler()
x = np.zeros([n,len(magnitude_names)-1])
for i in range(n):
    if target_ids[i] != target_ids2[i]:
        raise ValueError('target_ids dont match')
    for j in range(len(magnitude_names)-1):
        x[i,j] = magnitudes[i,j]-magnitudes[i,j+1]
x = scalar.fit_transform(x)

if m == 0 or m == 5:
    x = np.concatenate((ones,x),axis=1)

# assigning outcomes as EW (equivalent width) and getting their inverse variance
if loga:
    EW = np.log10(data_train[lines[l]][target_pos])
else:
    EW = data_train[lines[l]][target_pos]
ivar=data_train[lines[l]+"_IVAR"][target_pos]

## doing cross-validation by splitting data into N_cv intervals. I store all the outcomes in EW_fit_all, ivar_all, etc...
N_cv = 10
x_split = np.split(x,N_cv)
EW_split = np.split(EW,N_cv)
ivar_split = np.split(ivar,N_cv)

EW_fit_all = []
EW_obs_all = []
ivar_all = []

spearman_all = []
nmad_all = []
for i in range(N_cv):
    ## assigning the training and validation sets
    x_valid = x_split[i]
    EW_valid = EW_split[i]
    ivar_valid = ivar_split[i]
    x_to_combine = []
    EW_to_combine = []
    for j in range(N_cv):
        if j != i:
            x_to_combine.append(x_split[j])
            EW_to_combine.append(EW_split[j])
    x_train = np.concatenate(tuple(x_to_combine),axis=0)
    EW_train = np.concatenate(tuple(EW_to_combine),axis=0)

    # predicting EWs using LLR
    if m == 0:
        EW_fit,zeros = LLR.LLR(x_valid, x_train, EW_train, 100, 'inverse_distance')
        EW_valid = np.delete(EW_valid, zeros, axis=0)
        ivar_valid = np.delete(ivar_valid, zeros, axis=0)
    if m == 1:
        model = RandomForestRegressor(n_estimators=100)
        model.fit(x_train, EW_train)
        EW_fit = model.predict(x_valid)
    if m == 2:
        model = GradientBoostingRegressor(n_estimators=100)
        model.fit(x_train, EW_train)
        EW_fit = model.predict(x_valid)
    if m == 3:
        model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05)
        model.fit(x_train, EW_train, early_stopping_rounds=5, eval_set=[(x_valid,EW_valid)], verbose=False)
        EW_fit = model.predict(x_valid)
        print(model.best_ntree_limit)
    if m == 4:
        model_input = layers.Input(shape=x.shape[1])
        h1 = layers.Dense(units=150, kernel_initializer="he_normal")(model_input)
        a1 = layers.PReLU()(h1)
        h2 = layers.Dense(units=150, kernel_initializer="he_normal")(a1)
        a2 = layers.PReLU()(h2)
        h3 = layers.Dense(units=150, kernel_initializer="he_normal")(a2)
        a3 = layers.PReLU()(h3)
        output_layer = layers.Dense(1, activation='linear')(a3)
        model = keras.models.Model(inputs=model_input, outputs=output_layer)
        # model=keras.Sequential([
        #     layers.Dense(units=100, kernel_initializer="he_normal", activation='PReLU', input_shape=[x.shape[1]]),
        #     layers.Dense(units=100, kernel_initializer="he_normal", activation='PReLU'),
        #     layers.Dense(units=100, kernel_initializer="he_normal", activation='PReLU'),
        #     layers.Dense(units=1, activation="linear"),
        # ])
        model.compile(optimizer='Adam', loss='mse', metrics='mse')
        n_epochs = 100
        batch_size = 100
        history = model.fit(x_train, EW_train, batch_size=batch_size, epochs=n_epochs, verbose=0, validation_data=(x_valid, EW_valid))
        EW_fit = model.predict(x_valid)

    if m == 5:
        EW_fit,zeros = LLR.LLR_slow(x_valid, x_train, EW_train, 800, 'inverse_distance')
        EW_valid = np.delete(EW_valid, zeros, axis=0)
        ivar_valid = np.delete(ivar_valid, zeros, axis=0)

    if m ==6:
        kernel = RBF()
        gpr = GaussianProcessRegressor(kernel=kernel)
        gpr.fit(x_train, EW_train)
        EW_fit = gpr.predict(x_valid)

    # calculating spearman coefficient and nmad for fit.
    nmad = np.abs(EW_fit-EW_valid)

    EW_fit_all.append(EW_fit)
    EW_obs_all.append(EW_valid)
    ivar_all.append(ivar_valid)

    spearman_all.append(stats.spearmanr(EW_fit,EW_valid)[0])
    nmad_all.append(1.48*np.median(nmad))

print(lines[l])
print(spearman_all)
print("av spearman = "+str(np.average(spearman_all)))
print(nmad_all)
print("av nmad = "+str(np.average(nmad_all)))

print("\n")

# if loga:
#     np.savez_compressed(server_paths[server] + "ew_results/ugriz/m" +str(m)+ "/logEW_fit_ugriz_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)\
#                             +"_ML"+str(m)+".txt", EW_fit_all)
#     np.savez_compressed(server_paths[server] + "ew_results/ugriz/m" +str(m)+ "/logEW_obs_ugriz_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)\
#                             +"_ML"+str(m)+".txt", EW_obs_all)
#     np.savez_compressed(server_paths[server] + "ew_results/ugriz/m" +str(m)+ "/line_ivars_ugriz_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)\
#                             +"_ML"+str(m)+".txt", ivar_all)
    # else:
    #     np.savez_compressed(server_paths[server] + "results/EW_fit_classical_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)+"_model"+str(m)+".txt",EW_fit_all)
    #     np.savez_compressed(server_paths[server] + "results/EW_obs_classical_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)+"_model"+str(m)+".txt",EW_obs_all)
    #     np.savez_compressed(server_paths[server] + "results/EW_ivar_classical_selection"+str(run)+"_line"+str(lines[l])+"_bins"+str(N)+"_model"+str(m)+".txt",ivar_all)


36223
28520


/tmp/ipykernel_6800/2986895657.py:41: RuntimeWarning: invalid value encountered in multiply
  snr_all[:,0] = data_train[lines[0]]*np.sqrt(data_train[lines[0]+"_IVAR"])


KeyboardInterrupt: 

In [7]:
EW_fit_all[1]


IndexError: list index out of range

In [8]:
spearman_all[0]

0.8459659753644809